# Load Trained Attention Model
This notebook contains prototype code for loading a trained attention model.

In [42]:
import os
import tensorflow as tf

from komorebi.libs.dataset.types.dataset_config import DatasetConfiguration
from komorebi.libs.dataset.types.tf_dataset_wrapper import tf_dataset_wrapper 
from komorebi.libs.trainer.trainer_utils import compute_number_of_batches, get_data_stream_for_epoch

## Load dataset examples

In [43]:
# create dataset configuration
dataset_config = DatasetConfiguration(
    dataset_name='validation_dataset', 
    examples_directory='/Users/andy/Projects/attention_histone_modification/datasets/attention_validation_tf_dataset')

# instantiate dataset
dataset = tf_dataset_wrapper(dataset_config)

# build dataset iterator
dataset.build_one_shot_iterator()
data_stream_op = dataset.iterator.get_next()

# run iterator through session to get examples
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    data_stream_op = get_data_stream_for_epoch(dataset, sess)
    data = sess.run(data_stream_op)
    

AttributeError: 'tf_dataset_wrapper' object has no attribute 'build_one_shot_iterator'

In [17]:
sequence_data = data['sequence']
annotation_data = data['annotation']
label_data = data['label']

In [18]:
annotation_data.shape

(1, 75, 320)

## Load trained model

In [36]:
# specify directory of saved model
saved_model_dir_4 = "/tmp/attention_experiment_test/trained_model" #(4 iterations)
saved_model_dir_2 = "/tmp/attention_experiment_test_2_iters/trained_model" #(2 iterations)
saved_model_dir = saved_model_dir_4

In [37]:
# load graph
with tf.Session(graph=tf.Graph()) as prediction_sess:
    serving_tag = tf.saved_model.tag_constants.SERVING
    tf.saved_model.loader.load(sess=prediction_sess, tags=[serving_tag], export_dir=saved_model_dir)
    
    graph = tf.get_default_graph()
    
    # get ops
    # sequence_ph = graph.get_tensor_by_name("sequences_1:0") # this works
    sequence_ph = graph.get_tensor_by_name("model_inputs/sequences:0") # trying to see if this works
    feature_ph = graph.get_tensor_by_name("model_inputs/features:0")
    logit_op = graph.get_tensor_by_name("decode_lstm/logits:0")
    
    # make predictions
    logits1 = prediction_sess.run(logit_op, feed_dict={sequence_ph: sequence_data, feature_ph: annotation_data})
    logits2 = prediction_sess.run(logit_op, feed_dict={sequence_ph: sequence_data, feature_ph: annotation_data})

INFO:tensorflow:Restoring parameters from /tmp/attention_experiment_test/trained_model/variables/variables


INFO:tensorflow:Restoring parameters from /tmp/attention_experiment_test/trained_model/variables/variables


In [21]:
import numpy as np
np.array_equal(logits1, logits2)

True

## Ops inspection

2 iterations produces 126810 ops 

4 iterations produces 126812 ops

In [22]:
def find_substring(query, ops):
    matches = []
    for op in ops:
        if query in op:
            matches.append(op)
    return matches

In [35]:
import copy
ops_iter2 = copy.deepcopy([n.name for n in graph.as_graph_def().node])

# get ops
sequence_op = find_substring("sequence", ops)
feature_op = find_substring("feature", ops)
context_op = find_substring("context", ops)
logit_op = find_substring("logit", ops)
prediction_op = find_substring("prediction", ops)

In [38]:
ops_iter4 = copy.deepcopy([n.name for n in graph.as_graph_def().node])

In [39]:
print len(ops_iter2)
print len(ops_iter4)

126810
126812


In [41]:
d_two = {k: 0 for k in ops_iter2}
for o in ops_iter4:
    if o not in d_two:
        print o

IteratorGetNext_2
IteratorGetNext_3
